In [7]:
import pandas as pd
import numpy as np

In [8]:
# importar pacotes usados na seleção do modelo e na medição da precisão
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [30]:
BASE = 'afastamentos'
#BASE = 'cadastro'

In [31]:
df = pd.read_csv(BASE + '-nomes-sexo.csv', index_col=0, 
                 dtype={'PERC': np.float32, 'LETRAS': 'uint8'})

In [32]:
df.head(10)

,SEXO
PNOME,
MARIA,F
JOSE,M
ANTONIO,M
FRANCISCO,M
JOAO,M
CARLOS,M
ANA,F
PAULO,M
LUIZ,M


In [33]:
df['SEXO'] = df['SEXO'].map({'F': 1, 'M': 2, 'X': 0}).astype('uint8')
#df.loc[df['SEXO'].isnull(), 'SEXO'] = 2
#df['SEXO'] = df['SEXO'].astype('uint8')

In [34]:
df['REV'] = df.index
df['REV'] = df['REV'].apply(lambda x: x[::-1])

In [45]:
MAIOR_QTDE_LETRAS = 16 # fixado para o maior arquivo

for i in range(MAIOR_QTDE_LETRAS):
    df['L' + str(i)] = df['REV'].apply(lambda x: ord(x[i]) - 64 if len(x) > i else 0).astype('uint8')

In [90]:
#df.dtypes

In [47]:
df.head(10)

,SEXO,REV,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,,
MARIA,1,AIRAM,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,2,ESOJ,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,2,OINOTNA,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,2,OCSICNARF,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,2,OAOJ,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,2,SOLRAC,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0
ANA,1,ANA,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
PAULO,2,OLUAP,15,12,21,1,16,0,0,0,0,0,0,0,0,0,0,0
LUIZ,2,ZIUL,26,9,21,12,0,0,0,0,0,0,0,0,0,0,0,0


In [48]:
#df.describe()

In [49]:
# definir dados de entrada

X = df.drop(['SEXO', 'REV'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo

print('Forma dos dados originais:', X.shape, y.shape)

Forma dos dados originais: (9726, 16) (9726,)


In [50]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# create pipeline
#rfe = RFE(estimator=DecisionTreeClassifier(), n_features_to_select=5)
rfe = RFECV(estimator=DecisionTreeClassifier())
model = DecisionTreeClassifier()
pipeline = Pipeline(steps=[('s',rfe), ('m',model)])

# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(pipeline, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')

# report performance
print('Accuracy: %.3f (%.3f)' % (np.mean(n_scores)*100, np.std(n_scores)*100))

rfe.fit(X, y)

# summarize all features
for i in range(X.shape[1]):
  print('Column: %d [%s], Rank: %.3f %s' % (
      i, X.columns[i], rfe.ranking_[i],
      'Selected ' if rfe.support_[i] else ''))

Accuracy: 96.754 (0.545)


/opt/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Column: 0 [L0], Rank: 1.000 Selected 
Column: 1 [L1], Rank: 1.000 Selected 
Column: 2 [L2], Rank: 1.000 Selected 
Column: 3 [L3], Rank: 2.000 
Column: 4 [L4], Rank: 3.000 
Column: 5 [L5], Rank: 4.000 
Column: 6 [L6], Rank: 5.000 
Column: 7 [L7], Rank: 6.000 
Column: 8 [L8], Rank: 7.000 
Column: 9 [L9], Rank: 9.000 
Column: 10 [L10], Rank: 8.000 
Column: 11 [L11], Rank: 11.000 
Column: 12 [L12], Rank: 14.000 
Column: 13 [L13], Rank: 13.000 
Column: 14 [L14], Rank: 12.000 
Column: 15 [L15], Rank: 10.000 


In [41]:
#df = df[['PERC', 'L0', 'L1', 'L2', 'L3', 'L4', 'L5', 'SEXO']]
#df.drop(['QTDE'], axis=1, inplace=True)

In [51]:
X.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [52]:
y.head()

PNOME
MARIA        1
JOSE         2
ANTONIO      2
FRANCISCO    2
JOAO         2
Name: SEXO, dtype: uint8

In [53]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [54]:
from datetime import datetime

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [55]:
# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [56]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42, multi_class='auto', max_iter=1000, solver='liblinear', C=0.01) # 85.18
evaluate_model('LR', model)

params = dict(solver=['liblinear','lbfgs'], C=np.logspace(-3,3,7))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False) 
Cross-Validation Score: 84.94 (+/- 0.62) [ 3696 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.7s finished


In [48]:
# Support Vector Machine (SVM)
from sklearn.svm import SVC

model = SVC(random_state=42, C=10, gamma=0.001, kernel='rbf') # 89.49
evaluate_model('SVM', model)

#params = dict(C=np.logspace(-1,1,3)) # 87.90
#params = dict(gamma=np.logspace(-3,0,4)) # 89.46
#params = dict(kernel=['linear','rbf','poly','sigmoid']) # 89.46
params = dict(C=np.logspace(-1,1,3),
              gamma=np.logspace(-3,0,4),
              kernel=['linear','rbf','poly','sigmoid'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SVC(C=10, gamma=0.001, random_state=42) 
Cross-Validation Score: 86.42 (+/- 0.19) [355805 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  5.9min finished


In [49]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.001) # 85.27
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(C=np.logspace(-4,3,8))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearSVC(C=0.001, random_state=42) 
Cross-Validation Score: 79.34 (+/- 0.20) [12827 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.8s finished


In [57]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=1) # 87.45
evaluate_model('KNN', model)

params = dict(n_neighbors=[1,3,5,7,9,11])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                     weights='uniform') 
Cross-Validation Score: 85.64 (+/- 0.83) [ 7021 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    7.0s finished


In [58]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 84.45
evaluate_model('LDA', model)

params = dict(solver=['svd','lsqr','eigen'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001) 
Cross-Validation Score: 85.18 (+/- 0.64) [ 1502 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.5s finished


In [59]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001) 
Cross-Validation Score: 10.44 (+/- 0.71) [ 1022 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.0s finished


In [60]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=0.1) # 67.62
evaluate_model('GNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GaussianNB(priors=None, var_smoothing=0.1) 
Cross-Validation Score: 87.18 (+/- 0.20) [  815 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


In [61]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 43.37
evaluate_model('BNB', model)

params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) 
Cross-Validation Score: 44.21 (+/- 0.56) [  738 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


In [62]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17) # 96.37
evaluate_model('DT', model)

params = dict(criterion=['gini','entropy'], max_depth=[3,5,7,11,13,17,19])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=17,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best') 
Cross-Validation Score: 95.01 (+/- 0.71) [ 1249 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


In [56]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42, max_features='auto', n_estimators=500) # 93.86
evaluate_model('RF', model)

params = dict(n_estimators=[10,50,100,500], max_features=['auto','sqrt','log2'])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
/home/rodrigo/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


RandomForestClassifier(n_estimators=500, random_state=42) 
Cross-Validation Score: 96.77 (+/- 0.12) [93711 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.6min finished


In [63]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), n_estimators=13) # 96.41
evaluate_model('ABDT', model)

params = dict(n_estimators=[1,3,5,7,9,11,13])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=17,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=42,
                           

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    9.2s finished


In [58]:
# Ensemble - Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=42, max_depth=35, n_estimators=250, learning_rate=0.1) # 96.75
evaluate_model('GB', model)

params = dict(max_depth=[1,3,5,7,9], n_estimators=[5,50,250,500], learning_rate=[0.01,0.1,1,10,100])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GradientBoostingClassifier(max_depth=35, n_estimators=250, random_state=42) 
Cross-Validation Score: 99.36 (+/- 0.06) [889223 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 14.8min finished


In [64]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=500, tol=0.1) # 77.02
evaluate_model('SGD', model)

params = dict(max_iter=[100, 200, 350, 500, 1000], tol=[0.01, 0.1, 1.0])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=500, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.1,
              validation_fraction=0.1, verbose=0, warm_start=False) 
Cross-Validation Score: 78.00 (+/- 8.19) [ 5117 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.1s finished


In [65]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=500, tol=0.001) # 75.58
evaluate_model('PCT', model)

params = dict(max_iter=[100, 200, 350, 500, 750, 1000], tol=[0.1, 0.01, 0.001])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=500, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=42, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False) 
Cross-Validation Score: 77.46 (+/- 2.12) [ 2055 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.0s finished


In [61]:
# Multi-Layer Perceptron (MLP)
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=42, solver='lbfgs', alpha=1e-07, hidden_layer_sizes=(50, 100, 50)) # 88.67
evaluate_model('MLP', model)

params = dict(alpha=np.logspace(-9,-1,5), 
              solver=['lbfgs','sgd','adam'], 
              hidden_layer_sizes=[(50,50,50), (50,100,50), (100,)])
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


MLPClassifier(alpha=1e-07, hidden_layer_sizes=(50, 100, 50), random_state=42,
              solver='lbfgs') 
Cross-Validation Score: 88.16 (+/- 0.57) [282420 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  4.7min finished


In [66]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=9, min_child_weight=1, gamma=0.1, subsample=0.9, colsample_bytree=0.7, reg_alpha=1e-05) # 97.67
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 97.60
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 97.68
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 97.72
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 97.74
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0.1,
              learning_rate=0.1, max_delta_step=0, max_depth=9,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.9, verbosity=1) 
Cross-Validation Score: 96.29 (+/- 0.56) [74774 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.2min finished


In [67]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(num_leaves=73, max_depth=24, learning_rate=0.5, feature_fraction=0.75) # 97.51
evaluate_model('LGBM', model)

#params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 97.18
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 97.51
#params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 97.62
#fine_tune_model(model, params)

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier() # 97.04
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(iterations=[10,50,100,250,500,1000],
              learning_rate=[0.01,0.05,0.1,0.5],
              depth=range(1,11,2)) 
#fine_tune_model(model, params)

In [68]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
  (model, score, stdev, elapsed) = value
  names.append(key)
  estimators.append(model)
  scores.append(score)
  stdevs.append(stdev)
  times.append(elapsed)

In [69]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
10,XGB,96.288266,0.563784,74774,"XGBClassifier(base_score=0.5, booster='gbtree'..."
6,DT,95.013329,0.708934,1249,"DecisionTreeClassifier(class_weight=None, crit..."
7,ABDT,94.478730,0.541634,9204,"AdaBoostClassifier(algorithm='SAMME.R',\n ..."
4,GNB,87.178715,0.198164,815,"GaussianNB(priors=None, var_smoothing=0.1)"
1,KNN,85.636367,0.825470,7021,"KNeighborsClassifier(algorithm='auto', leaf_si..."
2,LDA,85.183983,0.644626,1502,"LinearDiscriminantAnalysis(n_components=None, ..."
0,LR,84.937207,0.622949,3696,"LogisticRegression(C=0.01, class_weight=None, ..."
8,SGD,77.996719,8.193170,5117,"SGDClassifier(alpha=0.0001, average=False, cla..."
9,PCT,77.462791,2.121366,2055,"Perceptron(alpha=0.0001, class_weight=None, ea..."
5,BNB,44.211452,0.561213,738,"BernoulliNB(alpha=1.0, binarize=0.0, class_pri..."


In [83]:
model = models['DT'][0]
model

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=17,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [84]:
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=17,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [85]:
y_pred = model.predict(X)
y_pred

array([1, 2, 2, ..., 0, 2, 2], dtype=uint8)

In [86]:
dados = pd.DataFrame({'REAL': y, 'PREV': y_pred}, index=X.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'X', 1: 'F', 2: 'M'})
dados.head()

,REAL,PREV
PNOME,,
MARIA,F,F
JOSE,M,M
ANTONIO,M,M
FRANCISCO,M,M
JOAO,M,M


In [87]:
from sklearn.metrics import accuracy_score

accuracy_score(y, y_pred)

0.999383096853794

In [88]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, y_pred)

array([[1015,    0,    0],
       [   0, 4373,    3],
       [   1,    2, 4332]], dtype=int64)

In [89]:
dados[dados['REAL'] != dados['PREV']]

,REAL,PREV
PNOME,,
LILIAN,F,M
FRANKLIN,M,X
LILLIAN,F,M
LEONIR,M,F
NAMIR,M,F
GEOVANI,F,M
